In [1]:
import pandas as pd
import pickle


In [15]:
df = pd.read_csv("../data/scrap/demanda_tiempo_real.csv")
df.drop(["percentage_Demanda real", "percentage_Demanda programada", "percentage_Demanda prevista"], axis = 1, inplace = True)
df["datetime"] = pd.to_datetime(df["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
df["drop"] = df["datetime"].where(df["datetime"].dt.minute % 10 == 0, other= "drop")
df = df[df["drop"] != "drop"]
df.drop(["drop"], inplace = True, axis = 1)
df = df[["datetime", "value_Demanda real"]].rename(columns = {"value_Demanda real" : "demanda_real"})

In [16]:
df.head()

,datetime,demanda_real
0,2013-12-31 23:00:00+00:00,24546
1,2013-12-31 23:10:00+00:00,24309
2,2013-12-31 23:20:00+00:00,24348
3,2013-12-31 23:30:00+00:00,24321
4,2013-12-31 23:40:00+00:00,24194
